# Scaling audio files to same DBFS value
Audio files were previously edited to remove leading and trailing silences
Metadata of each audio file, including its loudness level in dBFS were extracted and stored in an excel sheet. 
This metadata will be used to scale audio files to the same DBFS value. 

In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
from pydub import AudioSegment

/Users/jannaha/anaconda3/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
#Specify excel file of audiofiles metadata
metadata = '/Users/jannaha/Desktop/audio_files/output/metadata_delsilence.xlsx'

In [3]:
#folder path containing audio files that has been edited to remove silences
audio_folder = '/Users/jannaha/Desktop/audio_files/output'

In [4]:
#Specify excel file with audio filenames list
filenames = '/Users/jannaha/Desktop/audio_files/output/wavefiles_delsilence.xlsx'

In [5]:
df = pd.read_excel(metadata)
df.head()

,sample,channels,frames,rms,dbfs,peak_amp,total_len
0,2,1,44100,874,-31.478770,5686,519
1,2,1,44100,474,-36.793432,3154,1043
2,2,1,44100,789,-32.367459,5335,397
3,2,1,44100,869,-31.528603,4919,623
4,2,1,44100,519,-36.005652,3045,200


In [6]:
df2 = pd.read_excel(filenames)
df2.head()

,wavefile,duration,silence,total_len,dbfs_loudness
0,001 Saya 01.wav,657,"[[0, 73], [592, 657]]",519,-31.478770
1,002 kita 01.wav,1200,"[[0, 81], [1013, 1073], [1124, 1200]]",1043,-36.793432
2,003 kalau 01.wav,776,"[[119, 177], [574, 776]]",397,-32.367459
3,004 jadi 01.wav,766,"[[623, 766]]",623,-31.528603
4,005 tapi 01.wav,604,"[[184, 270], [470, 523]]",200,-36.005652


In [7]:
merged_df = pd.merge(df, df2, on='total_len')
merged_df.head()

,sample,channels,frames,rms,dbfs,peak_amp,total_len,wavefile,duration,silence,dbfs_loudness
0,2,1,44100,874,-31.478770,5686,519,001 Saya 01.wav,657,"[[0, 73], [592, 657]]",-31.478770
1,2,1,44100,474,-36.793432,3154,1043,002 kita 01.wav,1200,"[[0, 81], [1013, 1073], [1124, 1200]]",-36.793432
2,2,1,44100,789,-32.367459,5335,397,003 kalau 01.wav,776,"[[119, 177], [574, 776]]",-32.367459
3,2,1,44100,869,-31.528603,4919,623,004 jadi 01.wav,766,"[[623, 766]]",-31.528603
4,2,1,44100,519,-36.005652,3045,200,005 tapi 01.wav,604,"[[184, 270], [470, 523]]",-36.005652


In [8]:
#iterate through each row or audiofile to scale files to a fixed RMS or dbV value
def scale_files():
    for i,row in merged_df.iterrows():
        audio = merged_df.loc[i,'wavefile']
        audiofile = audio[:-4] + '-new.wav'
        audiofile_path = os.path.join(audio_folder, audiofile)
        
        if not os.path.isfile(audiofile_path):
            print(f"{audiofile_path} does not exist")
        
        #target_gain is the average RMS or dBV value previously calculated 
        target_gain = -31.817
        
        #RMS value of each audiofile before scaling
        current_dbfs = merged_df.loc[i, 'dbfs']
        
        #amount of gain to add/ subtract from each file to match to average RMS value
        gain_value = target_gain - float(current_dbfs)
        
        #scaling audio based on gain_value amount
        sound = AudioSegment.from_file(audiofile_path)
        old_dbfs = sound.dBFS
        
        scaled_audio = sound.apply_gain(gain_value)
        new_dbfs = scaled_audio.dBFS
        
        new_audiofile = audio[:-4] + 'scaled.wav'
        export_path = audio_folder + '/scaled/' + new_audiofile
        
        #export scaled audiofiles into new folder with path in export_path variable
        export_scaled = scaled_audio.export(export_path, format="wav")
    

In [9]:
scale_files()

# Creating an updated metadata of scaled files
Metadata will include the following:
1. Wavefile name
2. Sample
3. Channel
4. Frame size
5. RMS
6. Scaling factor applied
7. Old_RMS / Old_dBFS value
8. New_RMS / New_dBFS value  
 (_Note:This will also check if the newly scaled audiofiles are set to the right value_)
9. Total duration of audiofile 

In [16]:
# creating a new dataframe with updated metadata of scaled files
df3 = pd.DataFrame(columns = ['new_audiofile',
                              'samples',
                              'channels',
                              'frames',
                              'rms',
                              'old_dbfs',
                              'new_dbfs',
                              'total_len'])

In [17]:
for i,row in merged_df.iterrows():
    wavefile = merged_df.loc[i, 'wavefile'] #original filename
    old_audiofile = wavefile[:-4] + '-new.wav' #filenames with deleted silences
    new_audiofile = wavefile[:-4] + 'scaled.wav' #filenames scaled to same dBFS
    
    #specify filepath where deleted silences tracks are stored in
    oldaudio_path = os.path.join(audio_folder, old_audiofile)
    
    #specify filepath where scaled files are stored in
    newaudio_path = os.path.join(audio_folder, 'scaled', new_audiofile)
    
    #checks for errors in filepaths
    if not os.path.isfile(oldaudio_path):
            print(f"{oldaudio_path} does not exist")
    if not os.path.isfile(newaudio_path):
            print(f"{newaudio_path} does not exist")
    
    old_audio = AudioSegment.from_file(oldaudio_path)
    new_audio = AudioSegment.from_file(newaudio_path)

    sound = AudioSegment.from_file(newaudio_path)
    old_sound = AudioSegment.from_file(oldaudio_path)
        
    #Getting metadata to write to df3     
    samples = sound.sample_width
    channels = sound.channels
    frames = sound.frame_rate
    rms = sound.rms
    new_dbfs = sound.dBFS
    old_dbfs = old_sound.dBFS
    total_len = len(sound)
    
    #writing to df3 
    df3.loc[i] = [new_audiofile,
                  samples,
                  channels,
                  frames,
                  rms,
                  old_dbfs,
                  new_dbfs,
                  total_len]    

In [18]:
df3.head()

,new_audiofile,samples,channels,frames,rms,old_dbfs,new_dbfs,total_len
0,001 Saya 01scaled.wav,2,1,44100,841,-31.478770,-31.813079,519
1,002 kita 01scaled.wav,2,1,44100,841,-36.793432,-31.813079,1043
2,003 kalau 01scaled.wav,2,1,44100,840,-32.367459,-31.823413,397
3,004 jadi 01scaled.wav,2,1,44100,841,-31.528603,-31.813079,623
4,005 tapi 01scaled.wav,2,1,44100,840,-36.005652,-31.823413,200


In [20]:
#creating excel writer
write_to = '/Users/jannaha/Desktop/audio_files/output/scaled/metadata_scaled'

#writing dataframe to excelsheet, named sheet 1
df3.to_excel(write_to, index = False)